In [38]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import geopandas as gpd
from geodatasets import get_path
import os 

matplotlib.use("tkagg")

C:\Users\samue\DataspellProjects\covid analysis


In [41]:
nyc = gpd.read_file(get_path('nybb'))
url = "https://raw.githubusercontent.com/nychealth/coronavirus-data/master/totals/group-cases-by-boro.csv"
df_boro = pd.read_csv(url)

df_boro = df_boro[["BK_CASE_RATE", "BX_CASE_RATE", "MN_CASE_RATE", "QN_CASE_RATE", "SI_CASE_RATE"]]
df_boro = df_boro.loc[[0]]
print(df_boro)
# Melt the dataframe to convert columns to rows
melted_df = df_boro.melt(var_name='borough', value_name='case_rate')

# Clean up borough names
borough_mapping = {
    'BK_CASE_RATE': 'Brooklyn',
    'BX_CASE_RATE': 'Bronx',
    'MN_CASE_RATE': 'Manhattan',
    'QN_CASE_RATE': 'Queens',
    'SI_CASE_RATE': 'Staten Island'
}

melted_df['borough'] = melted_df['borough'].map(borough_mapping)

# Reset the index
melted_df = melted_df.reset_index(drop=True)
nyc.rename(columns={'BoroName':'borough'}, inplace=True)
bc_geo = nyc.merge(melted_df, on='borough')

fig, ax = plt.subplots(figsize=(12, 12))
# Plot the boroughs with a color scale based on confirmed cases
bc_geo.plot(column='case_rate', ax=ax, legend=True,
         legend_kwds={'label': 'Covid Case Rate (per 100,000)', 'orientation': 'vertical'},
         cmap='Reds', missing_kwds={'color': 'lightgrey'})

# Add borough labels
bc_geo.apply(lambda x: ax.annotate(text=x.borough, color='black', xy=x.geometry.centroid.coords[0],ha='center'), axis=1)

# Add title and adjust layout
plt.title('COVID-19 Case Rate by NYC Borough', fontsize=16)
plt.axis("off")
plt.tight_layout()

# Show the plot
plt.show()

   BK_CASE_RATE  BX_CASE_RATE  MN_CASE_RATE  QN_CASE_RATE  SI_CASE_RATE
0      41249.84      43468.76      40509.52      44517.62      53040.16


In [40]:
nyc = gpd.read_file(os.getcwd()+ "\\city-of-ny\\city-of-ny-i8iw-xf4u\\ZIP_CODE_040114.shp")
crs = {'init':'epsg:4326'}


url = "https://raw.githubusercontent.com/nychealth/coronavirus-data/master/totals/data-by-modzcta.csv"
df = pd.read_csv(url)
nyc["ZIPCODE"] = pd.to_numeric(nyc["ZIPCODE"])
df["ZIPCODE"] = df["MODIFIED_ZCTA"]

nyc = nyc.merge(df, on="ZIPCODE")


fig, ax = plt.subplots(figsize=(20,20))
nyc.plot(ax=ax, column="COVID_CASE_RATE", edgecolor="black",linewidth=0.25, cmap="Purples", legend=True,legend_kwds={'label': 'Covid Case Rate (per 100,000)', 'orientation': 'vertical'}, missing_kwds={'color': 'lightgrey'})

def calculate_angle(shape):
    coords = np.array(shape.exterior.coords)
    x_pts, y_pts = coords[:, 0], coords[:, 1]

    cov_matrix = np.cov(x_pts, y_pts)
    evals, evecs = np.linalg.eigh(cov_matrix)

    # Use the eigenvector corresponding to the largest eigenvalue
    main_axis = evecs[:, np.argmax(evals)]
    angle = np.arctan2(main_axis[1], main_axis[0])
    return np.degrees(angle)

# Function to adjust angle to prevent upside-down text
def adjust_angle(angle):
    if angle > 90:
        angle -= 180
    elif angle < -90:
        angle += 180
    return angle

# Add labels with size and rotation based on area and orientation
for idx, row in nyc.iterrows():
    shape = row.geometry
    zipcode = row['ZIPCODE']
    area = shape.area
    centroid = shape.centroid
    angle = calculate_angle(shape)
    adjusted_angle = adjust_angle(angle)
    text_size = np.sqrt(area) / 1000  # Adjust the factor for better visualization

    ax.annotate(zipcode, (centroid.x, centroid.y), fontsize=text_size,
                ha='center', va='center', rotation=adjusted_angle,
                rotation_mode='anchor', color="black")
    
plt.title('NYC Covid Case Rate by Zipcodes', fontsize=20)
plt.axis("off")
fig.subplots_adjust(hspace=0.0, wspace=0.0)
plt.tight_layout()
plt.savefig("map.png", dpi=1000)

